# Tesla Stock Analysis

## About this Project and Me

Hello, my name is Joseph Kemper. I am a Business Management student with an emphasis in Computer Programming. My goal behind choosing this project is to demonstrate what I am capable of and to build my skills for future projects. Future projects I have planned involve a large amount of data analysis and this project will serve as a good stepping stone to my long term goals. 

Goal of Analysis: Find what if any patterns that exist in Tesla Stock performance.

# Data Analysis

In [4]:
import pandas

# Load data into DataFrame
rawData = pandas.read_csv('RawData.csv')
rawData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2516 entries, 0 to 2515
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Date        2516 non-null   object
 1   Close/Last  2516 non-null   object
 2   Volume      2516 non-null   int64 
 3   Open        2516 non-null   object
 4   High        2516 non-null   object
 5   Low         2516 non-null   object
dtypes: int64(1), object(5)
memory usage: 118.1+ KB


## Data Exploration

In [7]:
percentiles = [.25,.5,.75,1]
include = [object, float, int]
rawData.describe(percentiles=percentiles,include=include)

,Date,Close/Last,Volume,Open,High,Low
count,2516,2516,2.516000e+03,2516,2516,2516
unique,2516,2450,NaN,2391,2377,2401
top,10/31/2023,$14.6387,NaN,$14.6667,$20.5333,$13.5333
freq,1,3,NaN,5,3,5
mean,NaN,NaN,1.141232e+08,NaN,NaN,NaN
std,NaN,NaN,7.681395e+07,NaN,NaN,NaN
min,NaN,NaN,1.065415e+07,NaN,NaN,NaN
25%,NaN,NaN,6.623726e+07,NaN,NaN,NaN
50%,NaN,NaN,9.307683e+07,NaN,NaN,NaN
75%,NaN,NaN,1.340551e+08,NaN,NaN,NaN
